In [1]:
import matplotlib.pyplot as plt
import jax.numpy as jnp
import numpy as np

from jaxlayerlumos import stackrt_eps_mu
from jaxlayerlumos import utils_materials

In [4]:

# Metadata
freq_range = (0.2, 2.0)  # GHz
inc_angle = 0.0         # degrees

# Frequency grid: use the metadata frequency range (converted to Hz)
frequencies = jnp.linspace(freq_range[0]*1e9, freq_range[1]*1e9, 100)
freq_ghz = np.array(frequencies) / 1e9  # for plotting in GHz

# Design configuration (first design line from CSV)
# Materials as given (first 5 numbers from CSV)
materials_data = ["14", "8", "4", "4", "4"]
# Thicknesses in mm (next 5 numbers from CSV)
thicknesses_mm = [0.966, 1.002, 1.182, 0.984, 1.380]
# Reference reflection value (dB) provided for comparison
ref_R_db = -32.52

# Augment the materials list with ambient "Air" and termination "PEC"
materials = ["Air"] + materials_data + ["PEC"]
# Convert thicknesses from mm to m and pad with 0 (for air and PEC)
thicknesses = [0.0] + [x * 1e-3 for x in thicknesses_mm] + [0.0]

print("Materials stack:", materials)
print("Layer thicknesses (m):", thicknesses)

print("frequencies: ", frequencies)
eps_stack, mu_stack = utils_materials.get_eps_mu(materials, frequencies)

# Compute reflection and transmission coefficients at normal incidence (0°)
R_TE, T_TE, R_TM, T_TM = stackrt_eps_mu(eps_stack, mu_stack, thicknesses, frequencies, inc_angle)

# Average the TE and TM polarizations
R_linear = (R_TE + R_TM) / 2.0
R_db = 10 * jnp.log10(R_linear).squeeze()  # Reflection in dB

# Identify the maximum reflection within the frequency grid
idx_max = np.argmax(np.array(R_db))
max_R_db = R_db[idx_max]
max_freq = freq_ghz[idx_max]
print(f"Maximum Reflection: {max_R_db:.2f} dB at {max_freq:.2f} GHz (Reference: {ref_R_db} dB)")

# --- Plotting ---
plt.semilogx(freq_ghz, np.array(R_db), label="Reflection (dB)")
plt.xlabel("Frequency (GHz)")
plt.ylabel("Reflection (dB)")
plt.title("RF Reflection vs Frequency for Hard-Coded Design")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()



Materials stack: ['Air', '14', '8', '4', '4', '4', 'PEC']
Layer thicknesses (m): [0.0, 0.000966, 0.001002, 0.0011819999999999999, 0.000984, 0.00138, 0.0]
frequencies:  [2.00000000e+08 2.18181818e+08 2.36363636e+08 2.54545455e+08
 2.72727273e+08 2.90909091e+08 3.09090909e+08 3.27272727e+08
 3.45454545e+08 3.63636364e+08 3.81818182e+08 4.00000000e+08
 4.18181818e+08 4.36363636e+08 4.54545455e+08 4.72727273e+08
 4.90909091e+08 5.09090909e+08 5.27272727e+08 5.45454545e+08
 5.63636364e+08 5.81818182e+08 6.00000000e+08 6.18181818e+08
 6.36363636e+08 6.54545455e+08 6.72727273e+08 6.90909091e+08
 7.09090909e+08 7.27272727e+08 7.45454545e+08 7.63636364e+08
 7.81818182e+08 8.00000000e+08 8.18181818e+08 8.36363636e+08
 8.54545455e+08 8.72727273e+08 8.90909091e+08 9.09090909e+08
 9.27272727e+08 9.45454545e+08 9.63636364e+08 9.81818182e+08
 1.00000000e+09 1.01818182e+09 1.03636364e+09 1.05454545e+09
 1.07272727e+09 1.09090909e+09 1.10909091e+09 1.12727273e+09
 1.14545455e+09 1.16363636e+09 1.181818

ValueError: could not convert string to float: 'wl'